# Constraint Expressions: Server-side Subsetting

## Motivation

Many OPeNDAP servers contain collections of datasets that in aggregation describe a complete dataset (e.g. a multi-year simulation). In some cases, each dataset may contain $\sim O(10-100)$ of variables in them -- this is particularly true for Level 2 data, see for example [this ATLAS03 dataset](http://test.opendap.org:8080/opendap/atlas03/ATL03_20200131234815_05540602_003_01.h5.dmr.html) from NASA. This means PyDAP must parse 100s of variables per file. PyDAP is a fast parser of *DMRs* (the [DAP4 metadata](https://opendap.github.io/dap4-specification/DAP4.html#_dmr_declarations) response), but:

1. PyDAP does not (yet) aggregate datasets nor URLs.
2. PyDAP does not make CF-checks, nor does it define label-based operations.

`xarray`, on the other hand, allows for parallel read and aggregation of multiple datasets. However, despite the awesome parallel features of `xarray`, it can be slow during the aggregation of multiple datasets because it performs a number of checks on the metadata so that the collection of files are "safe" to open and operate upon, according to the internal logics of `xarray`. This means that xarray makes extra checks to ensure safe label-based operations, and the time it takes can grow with the number of variables.

```{note}
Even though `xarray` hasa. method to drop variables from the virtual `xarray.Dataset`, it is only available **after** the dataset is created. This is, if you have 100s of files each with 100s of variables, but you are only interested in 2-3 variables per file, the client must parse and check all variables in all files, to create a dataset from which you will then drop all but the 2-3 variables of interest.
```

These, in combination, provide an obstacle for initial data exploration of collections of OPeNDAP datasets that contain many `variables`, `Groups`, among other complex types.


## Approach

One can add `Constraint Expresions` (`CEs`) to the dataset URL and sent it to the remote OPeNDAP server, to:

1. Request a subset of variables, and
2. Request a spatial subset of variables.


These `CEs` are increadible powerful from the user perspective because `pydap`, as a backend engine to `xarray`, can receive a much smaller DAP response from the remote OPeNDAP server. This is, the `CEs inform the remote OPeNDAP server to subset close to the data`. The response may be faster by $\sim O(1)$ second overall, but when considering a 100s of URLs, the result can have a significant impact on the performance and user experience.


Below we review `Constraint Expressions` with real data on OPeNDAP servers. Because there are two distinct DAP Protocols (see [Pydap as a client](client)), we will review the two cases separately.


In [ ]:
from pydap.client import open_url
import numpy as np
import matplotlib.pyplot as plt

## Constraint Expressions in DAP4

Here we demonstrate the use of CE in arrays of 1 and 2 dimensions, in two distinct datasets:

1. [ATLAS03](http://test.opendap.org:8080/opendap/atlas03/ATL03_20200131234815_05540602_003_01.h5.dmr.html), level 2 Data.
2. [Daily MUR Sea Surface Temperature](http://test.opendap.org:8080/opendap/ghrsst/20210102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.h5.dmr), level 4 data provided by PODAAC at JPL.


```{note}
For more interesting examples, take a look at [PACE example](notebooks/PACE)
```

### ATLAS03

This dataset has many nested Groups, with many variables in them. Say we are only interested in the variables:

1. `delta_time`.
2. `lat_ph`.
3. `lon_ph`.

The tree variables lie within the Group `heights`, which in turn is nested in the `Group` with name `gt3r`. 

Lets open the remote file naively, by requesting all the variables from the OPeNDAP server ([Hyrax](https://www.opendap.org/software/hyrax-data-server/) in this case)


In [ ]:
data_url = 'http://test.opendap.org:8080/opendap/atlas03/ATL03_20200131234815_05540602_003_01.h5'

In [ ]:
%%time
ds = open_url(data_url, protocol='dap4')

Below, we print the entire tree directory within the HDF5 dataset

In [ ]:
ds.tree()

We must note that the same variable names appear in other groups across the file, in the different other `Groups`. 

The DAP4 Protocol makes it easier to identify each variable defined within a `Group` with a unique [Fully Qualyfying Name](https://opendap.github.io/dap4-specification/DAP4.html#_fully_qualified_names). In this case, the FQN of each variable, similar to the navigating thgrough file system is:

| VarName | FQN_VarName |
| :- | -: | 
| `delta_time` | `/gt3r/heights/delta_time` |
| `lat_ph` | `/gt3r/heights/lat_ph` |
| `lon_ph` | `/gt3r/heights/lon_ph` |


With this knowledge, we want to only request these variables from the server's _DMR_. The DAP4 specirication for Constraint Expressions is:


$$
\text{Data_url + ?dap4.ce=<FQN_VarName1>;<FQN_VarName2>;<FQN_VarName3> }
$$

Let's try it now:


In [ ]:
data_urlCE = data_url+'?dap4.ce=/gt3r/heights/delta_time;/gt3r/heights/lat_ph;/gt3r/heights/lon_ph'

In [ ]:
%%time
ds = open_url(data_urlCE, protocol='dap4')

### This is an order of magnitude faster than before!

Because `Groups` are NOT part of the DAP2 data model, we cannot illustrate the CE in DAP2 with the `ATLAS03` dataset. Instead, we now look at the `COADS` dataset which has 2D arrays.



## Spatial Subsetting
Continuing to demonstrate `CEs` in the DAP4 data model, we now want to request only a subset of the variables. We first take a look at the complete dataset 












In [ ]:
%%time
data_url = 'http://test.opendap.org:8080/opendap/ghrsst/20210102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.h5'

In [ ]:
%%time
ds = open_url(data_url, protocol='dap4')

In [ ]:
ds.tree()

Same as before in the DAP4 data model, we have

| VarName | FQN_VarName |
| :- | -: | 
| `time` | `/time` |
| `lat` | `/lat` |
| `lon` | `/lon` |
| `mask` | `/mask` |
| `sea_ice_fraction` | `/sea_ice_fraction` |
| `dt_1km_data` | `/dt_1km_data` |
| `analysed_sst` | `/analysed_sst` |
| `analysis_error` | `/analysis_error` |
| `sst_anomaly` | `/sst_anomaly` |


In [ ]:
ds['analysed_sst'].shape

In [ ]:
ds['analysed_sst'].attributes

In [ ]:
print('Dimensions of variable:', ds['analysed_sst'].dimensions)

In [ ]:
ds.dimensions

We see that the variable has three dimensions, and these happen to coincide with those entire dataset.

### Spatial Subset

`We have yet to download/retrieve any data`. Only the _DMR_ has so far been requested by PyDAP. Consider the scenario where we only want to inspect a spatial subset of the data, defined by the `indexes ranges` aka `hyperslabs`. 

Say we want to retrieve only the first 100 points of `lat`, and the last 300 points of `lon`, of all variables in the dataset. In DAP4, there are two options to accomplish this with the URL.



1. `Traditional Approach`. Define the hyperslab for each variable you request. The [Data Request Form](http://test.opendap.org:8080/opendap/ghrsst/20210102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.h5.dmr) allows users to (interactively) build such URLS (by selecting boxes and modifying hyperslabs). Following this approach, you get the following Constrained URL

```
data_url + ?dap4.ce=/time;/lat[0:1:100];/lon[35700:1:];/mask[0][0:1:100][35700:1:];/sea_ice_fraction[0][0:1:100][35700:1:];/dt_1km_data[0][0:1:100][35700:1:];/analysed_sst[0][0:1:100][35700:1:];/analysed_sst[0][0:1:100][35700:1:];/analysis_error[0][0:1:100][35700:1:];/sst_anomaly[0][0:1:100][35700:1:]
```

Lets try it!


In [ ]:
%%time
nds = open_url(data_url+'?dap4.ce=/time[0];/lat[0:1:300];/lon[35699:1:35999];/mask[0][0:1:300][35699:1:35999];/sea_ice_fraction[0][0:1:300][35699:1:35999];/dt_1km_data[0][0:1:300][35699:1:35999];/analysed_sst[0][0:1:300][35699:1:35999];/analysis_error[0][0:1:300][35699:1:35999];/sst_anomaly[0][0:1:300][35699:1:35999]', protocol='dap4')

In [ ]:
nds

In [ ]:
nds['/analysed_sst'].shape

In [ ]:
nds['/lon'].shape

The spatial subsetting described above worked fine. It is very verbose and you have to explicitely set the size of all variables, if you want the resulting
subset dataset to be self-consistent. For example, it can be easy to define the spatial subset only on a variable and not the rest.






In [ ]:
%%time
nds = open_url(data_url+'?dap4.ce=/time;/lat;/lon;/mask;/sea_ice_fraction;/dt_1km_data;/analysed_sst[0][0:1:300][35699:1:35999]', protocol='dap4')

In [ ]:
nds['lat'].shape != nds['analysed_sst'].shape

In [ ]:
nds['lat'].shape, nds['analysed_sst'].shape

Next we provide an alternative approach that is less error prone, and makes use of [Shared Dimensions](https://opendap.github.io/dap4-specification/DAP4.html#_subsetting_and_shared_dimensions).

2. `Shared Dimensions`: This alternative approach can be used to define the spatial subsetting via the dimensions. The syntax is:

```
data_url + ?dap4.ce=<FQN_Dim1>=[subset];<FQN_Dim2>=[subset];<FQN_Var1>;<FQN_Var3>;<FQN_Var3>;...<FQN_VarN>
```

Where `<FQN_Var1>` may be the same as `<FQN_Dim1>`. 

In the syntax above, 

a) The subset is first defined on the dimensions by the `=` signs. Dimensions may be Global (at the root level), or within a `Group`. 

b) The user then defines the variables ot be included in the request by PyDAP. The subset defined in the previous space will be applied to the variable.



Using the example, the much simplified URL becomes:


In [ ]:
%%time
nds = open_url(data_url+"?dap4.ce=/lat=[0:1:300];/lon=[35699:1:35999];/time;/lat;/lon;/mask;/sea_ice_fraction;/dt_1km_data;/analysed_sst;/analysis_error;/sst_anomaly", protocol='dap4')

In [ ]:
nds.tree()

In [ ]:
nds['/analysed_sst'].shape == nds['sst_anomaly'].shape